In [7]:
import os
import pandas as pd
import plotly.express as px
import streamlit as st
import matplotlib.pyplot as plt
import numpy as np

In [2]:
# 폴더내에 
def read_xlsx_files():
	path = "./data"
	file_list = os.listdir(path)
	lst_xlsx = [file for file in file_list if file.endswith(".xlsx")]
	return lst_xlsx
	


# 데이터 파일 불러오기 및 전처리
def get_excelfile(p_file):
	df = pd.read_excel(
		io = f'./data/{p_file}',
		engine="openpyxl",
		sheet_name="실시간 학생수",
		skiprows=1,
		usecols="A:D",
		nrows=39
	)

	# '반'칼럼 nan값 삭제
	df = df.dropna(axis=0, subset=['반'])

	# 이덱스 리셋
	df = df.reset_index(drop='INDEX')

    # 인원 합계
	df['합계'] = df['남'] + df['여']
	return df

lst_xlsx = read_xlsx_files()

In [3]:
lst_xlsx

['220823.xlsx', '220820.xlsx', '220831.xlsx', '220919.xlsx']

In [39]:
df = get_excelfile('220823.xlsx').ffill()


In [24]:
df.head()

,학년,반,남,여,합계
0,1학년,1반,12,10,22
1,1학년,2반,12,13,25
2,1학년,3반,13,12,25
3,1학년,4반,13,12,25
4,1학년,5반,11,13,24


## 220928_파일의 학년별 합계를 라인그래프로 나타내기
- 파일 읽어오기
- 데이터 프레임으로 변환
- groupby로 학년별 합계 모으기(리스트)
- 리스트를 데이터 프레임으로 만들기
- 라인 그래프로 표현하기

In [25]:
lst_xlsx = read_xlsx_files()

In [26]:
lst_xlsx

['220823.xlsx', '220820.xlsx', '220831.xlsx', '220919.xlsx']

In [46]:
FILE_SRC = '220823.xlsx'
df = get_excelfile(FILE_SRC).ffill()

df_0927 = df[['학년', '합계']]
t_0927 = df_0927.groupby('학년').sum()
test_t = t_0927.T.reset_index()
test_t = test_t[['1학년', '2학년', '3학년', '4학년', '5학년', '6학년']]
test_t.name = None
test_t

학년,1학년,2학년,3학년,4학년,5학년,6학년
0,121,100,118,144,138,137


In [47]:
test_t.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 1 entries, 0 to 0
Data columns (total 6 columns):
 #   Column  Non-Null Count  Dtype
---  ------  --------------  -----
 0   1학년     1 non-null      int64
 1   2학년     1 non-null      int64
 2   3학년     1 non-null      int64
 3   4학년     1 non-null      int64
 4   5학년     1 non-null      int64
 5   6학년     1 non-null      int64
dtypes: int64(6)
memory usage: 176.0 bytes


In [37]:
test_t.reset_index()

학년,index,1학년,2학년,3학년,4학년,5학년,6학년
0,0,121,100,118,144,138,137


In [60]:
for i in lst_xlsx:
    print(i.split('.')[0])

220823
220820
220831
220919


In [62]:
def make_list_by_date(p_file):
    name_date = p_file.split('.')[0]
    df = get_excelfile(p_file).ffill()

    df_0927 = df[['학년', '합계']]
    t_0927 = df_0927.groupby('학년').sum()
    test_t = t_0927.T.reset_index()
    test_t = test_t[['1학년', '2학년', '3학년', '4학년', '5학년', '6학년']]
    return test_t, name_date

In [102]:
result = []
lst_date = []
for i in lst_xlsx:
    df, name_date = make_list_by_date(i)
    result.append(df)
    lst_date.append(name_date)
df_result = pd.concat(result)
df_result['날짜'] = lst_date
df_result.columns.name = None
df_result = df_result.set_index('날짜')
df_result

,1학년,2학년,3학년,4학년,5학년,6학년
날짜,,,,,,
220823,121,100,118,144,138,137
220820,119,100,118,143,137,136
220831,121,99,117,144,138,138
220919,122,99,118,144,139,140


In [103]:
st.line_chart(df_result)

DeltaGenerator(_root_container=0, _provided_cursor=None, _parent=None, _block_type=None, _form_data=None)

In [93]:
df_result.columns.name = None

df_result.set_index('날짜').reset_index()

In [76]:
df_result = df_result.set_index(['날짜'])

In [77]:
st.line_chart(df_result)

DeltaGenerator(_root_container=0, _provided_cursor=None, _parent=None, _block_type=None, _form_data=None)

## 이전 작업

In [37]:
test = df.groupby('학년').sum()

test_t = test.T.reset_index()
test_t.columns = ['성별', '1학년', '2학년', '3학년', '4학년', '5학년', '6학년']
test_t = test_t.drop(index=2)

col_list = list(test_t)
col_list.remove('성별')
test_t['합계'] = test_t[col_list].sum(axis=1)
test_t

,성별,1학년,2학년,3학년,4학년,5학년,6학년,합계
0,남,61,55,64,77,72,76,405
1,여,60,45,54,67,66,61,353


In [39]:
test_t[['성별','합계']]

,성별,합계
0,남,405
1,여,353


In [40]:
test_t

,성별,1학년,2학년,3학년,4학년,5학년,6학년,합계
0,남,61,55,64,77,72,76,405
1,여,60,45,54,67,66,61,353


## 1,2학년 현황 만들기

In [13]:
df.head()

,학년,반,남,여,합계
0,1학년,1반,12,10,22
1,1학년,2반,12,13,25
2,1학년,3반,13,12,25
3,1학년,4반,13,12,25
4,1학년,5반,11,13,24


In [19]:
str_expr = "학년 == '1학년'"
df_1gr = df.query(str_expr)
df_1gr

,학년,반,남,여,합계
0,1학년,1반,12,10,22
1,1학년,2반,12,13,25
2,1학년,3반,13,12,25
3,1학년,4반,13,12,25
4,1학년,5반,11,13,24


In [24]:
test = df_1gr[['반','합계']]

In [ ]:
test = df.groupby('학년').sum()

test_t = test.T.reset_index()
test_t.columns = ['성별', '1학년', '2학년', '3학년', '4학년', '5학년', '6학년']
test_t = test_t.drop(index=2)

col_list = list(test_t)
col_list.remove('성별')
test_t['합계'] = test_t[col_list].sum(axis=1)
test_t

In [13]:
df.head()

,학년,반,남,여,합계
0,1학년,1반,12,10,22
1,1학년,2반,12,13,25
2,1학년,3반,13,12,25
3,1학년,4반,13,12,25
4,1학년,5반,11,13,24


In [23]:
test_t.columns

Index(['1학년', '2학년', '3학년', '4학년', '5학년', '6학년'], dtype='object', name='학년')

In [21]:
test_t.columns

Index(['index', '1학년', '2학년', '3학년', '4학년', '5학년', '6학년'], dtype='object', name='학년')

In [73]:
pd.DataFrame(
    np.random.randn(20, 3),
    columns=['a', 'b', 'c'])

,a,b,c
0,0.415287,1.252303,0.487601
1,0.955728,0.746735,-0.213883
2,-0.325175,-1.801130,0.298567
3,1.647392,-0.386948,-0.534729
4,-0.799192,0.805231,0.311832
5,0.753529,-0.564965,1.598073
6,0.314279,1.504978,1.536028
7,0.114740,-2.208057,-1.115367
8,-1.189036,-0.284718,-1.093461
9,1.300939,0.179993,1.936261


In [101]:
chart_data = pd.DataFrame(
    np.random.randn(20, 3),
    columns=['a', 'b', 'c'])

st.line_chart(chart_data)

DeltaGenerator(_root_container=0, _provided_cursor=None, _parent=None, _block_type=None, _form_data=None)